In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from random import randrange
from scipy.sparse import csr_matrix,vstack
from collections import Counter
import numpy as np
import scipy

In [2]:
# K = no of cluster
# d = distance measurement - 'c' for cosine, 'e' for euclidean
def hard_kmeans(K, d, data, labels):
    max_iterations = 200
    centroids = []        # List of centroids
    cluster = {}          # {index of centroid: [mnist samples]}
    cluster_labels = {}   # {index of centroid: [mnist lables]}
    cluster_index = {}    # {index of centroid: [mnist indices]}
    iterations = 0
    N = data.shape[0]
    
    # Assume K centroids (by random)
    for i in range(K):
        centroids.append(data[randrange(0, N - 1)])

    print('K=%d' % (K))
    
    while(1):
        
        # Convert list of csr matrices to a 2d csr matrix
        centroids = vstack(centroids) if type(data).__name__ == 'csr_matrix' else centroids
        
        # Compute distance matrix
        if d == 'c':
            distance_matrix = cosine_distances(data, centroids)
        else:
            distance_matrix = euclidean_distances(data, centroids)

        # E step - compute memberships given centroids
        cluster = {}
        cluster_labels = {}
        cluster_index = {}
        for i in range(N):
            cluster.setdefault(np.argmin(distance_matrix[i]), []).append(data[i])
            cluster_labels.setdefault(np.argmin(distance_matrix[i]), []).append(labels[i])
            cluster_index.setdefault(np.argmin(distance_matrix[i]), []).append(i)

        # Store the current centroids before the M step
        prev_centroids = []
        for k in cluster:
            prev_centroids.append(centroids[k])

        # M step - compute centroids given memberships
        centroids = []
        for k in cluster:
            # np.mean(vstack(cluster[k]), axis=0) returns a number, csr_matrix() converts it to a csr matrix
            centroids.append(csr_matrix(np.mean(vstack(cluster[k]), axis=0))) if type(data).__name__ == 'csr_matrix' else centroids.append(np.mean(cluster[k], axis=0))

        iterations += 1
        
        # Termination conditions - on convergence, else after a fixed number of iterations 
        if np.array_equal(centroids, prev_centroids): #allclose , atol=1e-2
            print('Iteration', iterations,': CONVERGED!')
            break

        if iterations == max_iterations:
            print('Iteration', iterations,': max reached')
            break
            
    # Store list of labels as a Counter
    for key,value in cluster_labels.items():
        cluster_labels[key] = Counter(value)

    # Calculate purity
    purity = 0
    for cluster in cluster_labels:
        purity += max(cluster_labels[cluster].values())
    purity /= N
    
    # Calculate gini index
    gini_index = 0
    for key,value in cluster_labels.items():
        gini = 0
        for k,v in value.items():
            gini += (v / sum(cluster_labels[key].values())) ** 2
        gini_index += 1 - gini
    gini_index /= K
    
    # Final result
    print('Purity -', round(purity, 4), 'Gini Index -', round(gini_index, 4), '\n')

In [3]:
# Fetch data
mnist_dataset = fetch_mldata('mnist original')

# Data and labels
mnist_data = mnist_dataset.data
mnist_labels = mnist_dataset.target

print(mnist_data.shape)
print(mnist_labels.shape)

(70000, 784)
(70000,)


In [4]:
k_list = [5, 10, 20]

print('Without Normalizing')
print('-------------------------------------------------')
print('Using Euclidean distances ....')
print('-------------------------------------------------')
for k in k_list:
    hard_kmeans(k, d='e', data=mnist_data, labels=mnist_labels)

print('-------------------------------------------------')
print('Using Cosine distances ....')
print('-------------------------------------------------')
for k in k_list:
    hard_kmeans(k, d='c', data=mnist_data, labels=mnist_labels)

Without Normalizing
-------------------------------------------------
Using Euclidean distances ....
-------------------------------------------------
K=5
Iteration 36 : CONVERGED!
Purity - 0.3901 Gini Index - 0.7282 

K=10
Iteration 44 : CONVERGED!
Purity - 0.5982 Gini Index - 0.4653 

K=20
Iteration 74 : CONVERGED!
Purity - 0.7146 Gini Index - 0.3622 

-------------------------------------------------
Using Cosine distances ....
-------------------------------------------------
K=5
Iteration 75 : CONVERGED!
Purity - 0.3933 Gini Index - 0.7044 

K=10
Iteration 122 : CONVERGED!
Purity - 0.6018 Gini Index - 0.4887 

K=20
Iteration 66 : CONVERGED!
Purity - 0.7307 Gini Index - 0.3705 



# With normalizing

In [5]:
# Normalize data
norm_mnist_data = np.divide(mnist_data, 255)
print(norm_mnist_data.shape)

(70000, 784)


In [6]:
k_list = [5, 10, 20]

print('With Normalizing')
print('-------------------------------------------------')
print('Using Euclidean distances ....')
print('-------------------------------------------------')
for k in k_list:
    hard_kmeans(k, d='e', data=norm_mnist_data, labels=mnist_labels)

print('-------------------------------------------------')
print('Using Cosine distances ....')
print('-------------------------------------------------')
for k in k_list:
    hard_kmeans(k, d='c', data=norm_mnist_data, labels=mnist_labels)

With Normalizing
-------------------------------------------------
Using Euclidean distances ....
-------------------------------------------------
K=5
Iteration 64 : CONVERGED!
Purity - 0.39 Gini Index - 0.7283 

K=10
Iteration 55 : CONVERGED!
Purity - 0.5914 Gini Index - 0.4955 

K=20
Iteration 144 : CONVERGED!
Purity - 0.7047 Gini Index - 0.352 

-------------------------------------------------
Using Cosine distances ....
-------------------------------------------------
K=5
Iteration 37 : CONVERGED!
Purity - 0.4641 Gini Index - 0.6118 

K=10
Iteration 187 : CONVERGED!
Purity - 0.6037 Gini Index - 0.4783 

K=20
Iteration 82 : CONVERGED!
Purity - 0.7304 Gini Index - 0.3748 

